# Definición del proyecto de datos

Todo este proyecto tiene su base en la obtención de ciertos insights a partir de 3 bases de datos, las cuales tiene como cobertura geográfica el estado de los ángeles de estados unidos, es decir, es un proyecto donde en particular se estudiara las relaciones entre las diferentes bases de datos donde la región a estudiar son los ángeles.

- Crimen

- Negocios

- Vivienda

El contexto, es simple, trabajamos para el gobierno como alcalde de la ciudad y por tanto, las decisiones que se tomen, deberán de ser tomadas con una perspectiva meramente política, por ello es que los objetivos de este análisis deben de centrarse en la obtención de accionables claros a partir de las relaciones e interacciones encontradas en las bases de datos.

Los ángeles se dividen en 114 vecindarios, sobre los cuales se deberán de realizar el análisis, es decir, la investigación es a nivel de vecindarios. Por tanto, un punto relevante es evaluar la granularidad de la información en las bases de datos así como preguntas de la siguiente naturaleza

¿Qué vecindarios tienen una mayor tasa de muerte de negocios?, ¿se relaciona con la tasa de crimen?
¿Qué vecindarios tienen la mayor tasa de crimen?, ¿necesitan mayor atención policial?
¿Está esto afectando a los precios de las propiedades?  

In [1]:
import pandas as pd 
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd                # Manejo de datos geoespaciales :contentReference[oaicite:8]{index=8}
from shapely.geometry import Point     # Construcción de geometrías puntuales :contentReference[oaicite:9]{index=9}

import requests
import io
from http.client import IncompleteRead

from sklearn.neighbors import BallTree

import statsmodels.api as sm
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.oneway import test_scale_oneway
from statsmodels.stats.stattools import jarque_bera
from statsmodels.stats.nonparametric import rank_compare_2indep

In [2]:
sns.set_style("darkgrid")
pd.options.display.float_format = "{:,.2f}".format

In [3]:
# URLs de descarga
crime_url    = "https://data.lacity.org/api/views/2nrs-mtv8/rows.csv?accessType=DOWNLOAD"      # Crimen 2020–Presente :contentReference[oaicite:10]{index=10}
business_url = "https://data.lacity.org/api/views/r4uk-afju/rows.csv?accessType=DOWNLOAD"     # Listado de negocios :contentReference[oaicite:11]{index=11}
neighborhoods_url = "https://data.lacity.org/api/views/2nrs-mtv8/rows.csv?accessType=DOWNLOAD" # Vecindarios :contentReference[oaicite:12]{index=12} #https://geohub.lacity.org/datasets/d6c55385a0e749519f238b77135eafac_0.geojson
# zhvi_url     = "~/work/Neighborhood_zhvi.csv"
zhvi_url     = "./resources/Neighborhood_zhvi.csv"# ZHVI mensual por vecindario :contentReference[oaicite:12]{index=12}
areas_url    = "https://geohub.lacity.org/datasets/691805703915458da4b35d8088f29501_0.geojson" # Área por vecindario :contentReference[oaicite:13]{index=13}
bounds_url   = "https://geohub.lacity.org/datasets/d6c55385a0e749519f238b77135eafac_0.geojson" # Límites de vecindario :contentReference[oaicite:14]{index=14}
census_url   = "https://data.lacity.org/api/views/nwj3-ufba/rows.csv?accessType=DOWNLOAD"      # Censo 2010 por consejo :contentReference[oaicite:15]{index=15}


In [4]:
def fetch_csv_stream(url: str, timeout: int = 60) -> io.StringIO:
    """
    Descarga un CSV por streaming.  
    En caso de IncompleteRead, reutiliza los bytes parciales descargados.
    """
    try:
        resp = requests.get(url, stream=True, timeout=timeout)
        resp.raise_for_status()
        # Leer todo el texto (posible IncompleteRead aquí)
        text = resp.text
    except IncompleteRead as e:
        # e.partial es un bytes con lo descargado antes de la excepción
        print('Incompleto:', e.partial)
        text = e.partial.decode('utf-8', errors='replace')
    return io.StringIO(text)

# Crimen

## Definición de requerimientos de datos

En este primer paso se abordar el tratamiento de los datos que corresponden al crimen.

El primer paso para abordar el tratamiento de los datos que corresponden al crimen es comenzar por la definición de los requerimientos de datos. Este paso, se abordarán los temas relacionados con las fuentes, variables y objetivos de negocio para el análisis, estableciendo criterios de calidad y formatos esperados 

| Campo             | Descripción                                                                                                 |
|-------------------|-------------------------------------------------------------------------------------------------------------|
| **DR_NO**         | Número de expediente, formado por 2 dígitos de año + código de área + dígitos secuenciales                  |
| **Date Rptd**     | Fecha en la que se reportó el incidente al LAPD (MM/DD/YYYY)                                                |
| **DATE OCC**      | Fecha en que ocurrió el incidente (MM/DD/YYYY)                                                              |
| **TIME OCC**      | Hora de ocurrencia, en formato militar (24 h)                                                               |
| **AREA**          | Código de área (1–21) que identifica la estación comunitaria                                                |
| **AREA NAME**     | Nombre de la estación (“77th Street”, “Central”, etc.)                                                      |
| **Rpt Dist No**   | Número de distrito, subdivisión numérica dentro de cada área para análisis estadístico                       |
| **Part 1‑2**      | Indica si el delito es UCR Part I (crímenes más graves) o Part II                                            |
| **Crm Cd**        | Código de delito, código numérico principal asignado al incidente                                             |
| **Crm Cd Desc**   | Descripción textual del delito (`Crm Cd`)                                                                    |
| **MOCodes**       | Modus operandi: lista de códigos que describen comportamientos del sospechoso                                 |
| **Vict Age**      | Edad de la víctima, en años                                                                                  |
| **Vict Sex**      | Sexo de la víctima: M=Masculino, F=Femenino, X=Desconocido                                                   |
| **Vict Descent**  | Ascendencia/etnicidad de la víctima (H=Hispano, B=Negro, W=Blanco, A=Asiático, etc.)                         |
| **Premis Cd**     | Código de la ubicación (tipo de inmueble, vehículo o lugar)                                                  |
| **Premise Desc**  | Descripción de la ubicación (`Premis Cd`)                                                                    |
| **Weapon Used Cd**| Código del arma o instrumento utilizado                                                                       |
| **Weapon Desc**   | Descripción del arma (`Weapon Used Cd`)                                                                      |
| **Status**        | Estado del caso (p. ej. IC=In Commission, CA=Cleared by Arrest, etc.)                                         |
| **Status Desc**   | Descripción del código de estado (`Status`)                                                                  |
| **Crm Cd 1**      | Delito principal (más grave) del incidente                                                                   |
| **Crm Cd 2**      | Delito secundario 1 (si aplica)                                                                              |
| **Crm Cd 3**      | Delito secundario 2 (si aplica)                                                                              |
| **Crm Cd 4**      | Delito secundario 3 (si aplica)                                                                              |
| **LOCATION**      | Ubicación aproximada (número de manzana redondeado al centenar más cercano)                                  |
| **Cross Street**  | Calle transversal más cercana al punto de `LOCATION`                                                         |
| **LAT**           | Latitud en coordenadas geográficas (WGS 84) del punto del incidente                                           |
| **LON**           | Longitud en coordenadas geográficas (WGS 84) del punto del incidente                                          |


In [19]:
crime_buffer = fetch_csv_stream(crime_url)

raw_df = pd.read_csv(crime_buffer, low_memory=False) 

del crime_buffer

In [22]:
raw_df["Date Rptd"] = pd.to_datetime(raw_df["Date Rptd"], format="%m/%d/%Y %I:%M:%S %p", errors="raise")
raw_df["DATE OCC"] = pd.to_datetime(raw_df["DATE OCC"], format="%m/%d/%Y %I:%M:%S %p", errors="raise")

In [25]:
raw_df.columns = raw_df.columns.str.strip()

# Mostrar las primeras columnas para verificar nombres exactos
print(raw_df.columns.tolist())

['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME', 'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes', 'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT', 'LON']


In [19]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005199 entries, 0 to 1005198
Data columns (total 28 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   DR_NO           1005199 non-null  int64         
 1   Date Rptd       1005199 non-null  datetime64[ns]
 2   DATE OCC        1005199 non-null  datetime64[ns]
 3   TIME OCC        1005199 non-null  int64         
 4   AREA            1005199 non-null  int64         
 5   AREA NAME       1005199 non-null  object        
 6   Rpt Dist No     1005199 non-null  int64         
 7   Part 1-2        1005199 non-null  int64         
 8   Crm Cd          1005199 non-null  int64         
 9   Crm Cd Desc     1005199 non-null  object        
 10  Mocodes         853440 non-null   object        
 11  Vict Age        1005199 non-null  int64         
 12  Vict Sex        860418 non-null   object        
 13  Vict Descent    860406 non-null   object        
 14  Premis Cd       10

In [22]:
raw_df.isnull().sum()/len(raw_df)

DR_NO            0.00
Date Rptd        0.00
DATE OCC         0.00
TIME OCC         0.00
AREA             0.00
AREA NAME        0.00
Rpt Dist No      0.00
Part 1-2         0.00
Crm Cd           0.00
Crm Cd Desc      0.00
Mocodes          0.15
Vict Age         0.00
Vict Sex         0.14
Vict Descent     0.14
Premis Cd        0.00
Premis Desc      0.00
Weapon Used Cd   0.67
Weapon Desc      0.67
Status           0.00
Status Desc      0.00
Crm Cd 1         0.00
Crm Cd 2         0.93
Crm Cd 3         1.00
Crm Cd 4         1.00
LOCATION         0.00
Cross Street     0.85
LAT              0.00
LON              0.00
dtype: float64

## Perfilado de los datos

In [28]:
raw_df["DATE OCC"].min(), raw_df["DATE OCC"].max()

(Timestamp('2020-01-01 00:00:00'), Timestamp('2025-04-01 00:00:00'))

In [31]:
raw_df["AREA"].value_counts().index.max()

21

In [34]:
group_by_area = raw_df.groupby("AREA")["AREA NAME"].count()
group_by_area

AREA
1     69673
2     46826
3     57512
4     37097
5     41462
6     52430
7     48240
8     45731
9     42883
10    42157
11    42962
12    61762
13    49181
14    59521
15    51107
16    33136
17    41766
18    49941
19    40365
20    50071
21    41376
Name: AREA NAME, dtype: int64